# SQL + Pandas

In this section we will cover a step-by-step example to integrate the energy production data stored in a database (SQLite) and then using Python to analyze and plot the data. This guide walks through each step, from loading CSV data into an SQLite database to querying and plotting the energy production data for different countries.

This sections includes downloading the data from individual `.csv` files, loading them into a SQLite database, and visualizing the energy production using Python.

This guide shows how to:
1. Download CSV files from Eurostat.
2. Load data into a SQLite database.
3. Use Python to query and visualize the data.

## Step 1: Download the Data from Eurostat

The datasets are available on the Eurostat website under [Energy Statistics](https://ec.europa.eu/eurostat/web/energy). Let’s assume the data files have been downloaded and stored in your `~/Downloads` folder as `.csv` files.

For this example, we are using the following datasets:

In [3]:
import numpy as np
import pandas as pd
import sqlite3
import os

In [4]:
file_paths = {
    'coal': '../data/section4/euroStat/nrg_cb_pem_page_spreadsheet_coal.xlsx',
    'nonRenewables': '../data/section4/euroStat/nrg_cb_pem_page_spreadsheet_combustionFuels_nonRenewables.xlsx',
    'renewables': '../data/section4/euroStat/nrg_cb_pem_page_spreadsheet_combustionFuels_Renewables.xlsx',
    'geothermal': '../data/section4/euroStat/nrg_cb_pem_page_spreadsheet_geothermal.xlsx',
    'hydro': '../data/section4/euroStat/nrg_cb_pem_page_spreadsheet_hydro.xlsx',
    'naturalGas': '../data/section4/euroStat/nrg_cb_pem_page_spreadsheet_naturalGas.xlsx',
    'nuclear': '../data/section4/euroStat/nrg_cb_pem_page_spreadsheet_nuclear.xlsx',
    'oil': '../data/section4/euroStat/nrg_cb_pem_page_spreadsheet_oil.xlsx',
    'otherRenewables': '../data/section4/euroStat/nrg_cb_pem_page_spreadsheet_otherRenewables.xlsx',
    'solar': '../data/section4/euroStat/nrg_cb_pem_page_spreadsheet_solar.xlsx',
    'wind': '../data/section4/euroStat/nrg_cb_pem_page_spreadsheet_wind.xlsx'
}

In [5]:
# Connect to SQLite database (it will be created if it doesn't exist)
db_path = 'coal_energy_data.db'

#### Step 1: Check and Create the SQLite Database

Before beginning, the code checks if the SQLite database (coal_energy_data.db) already exists. If it does, the existing database file is deleted to ensure you are working with a fresh database.

In [6]:
# Check if the database file exists
if os.path.exists(db_path):
    print(f"Database {db_path} already exists. Deleting it...")
    os.remove(db_path)  # Delete the existing database file
    print(f"Database {db_path} has been deleted.")

Database coal_energy_data.db already exists. Deleting it...
Database coal_energy_data.db has been deleted.


In [7]:
# Now create a new SQLite connection
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

Afterward, the database is recreated by establishing a connection to the SQLite file.

#### Step 2: Load Data from Excel into SQLite

The coal energy data is loaded from an Excel file into a Pandas DataFrame. The raw data is then stored in the SQLite database as the initial step.

In [8]:
# Load the coal dataset
euroStat_coal = pd.read_excel(file_paths['coal'], sheet_name='coal', skiprows=range(0, 8))

In [9]:
# Store the raw data in the SQLite database
euroStat_coal.to_sql('euroStat_coal', conn, if_exists='replace', index=False)

50

#### Step 3: Remove the First Two Rows

Since the first two rows contain irrelevant information, such as metadata or unnecessary headers, they are removed using an SQL DELETE operation based on ROWID.

In [10]:
# Remove the first two rows (index 0 and 1)
cursor.execute("DELETE FROM euroStat_coal WHERE ROWID IN (1, 2);")
conn.commit()

#### Step 4: Drop Unnecessary Columns

Some columns in the data may have names starting with 'Unnamed', which indicate they are placeholders. Since SQLite does not directly support dropping columns, the workflow selects only the necessary columns and recreates a new table.

In [11]:
# Drop columns where the name starts with 'Unnamed'
cursor.execute("PRAGMA table_info(euroStat_coal);")
columns_info = cursor.fetchall()
columns_to_keep = [col[1] for col in columns_info if not col[1].startswith('Unnamed')]

In [12]:
# Create a new table with only the required columns
cursor.execute(f"""
    CREATE TABLE euroStat_coal_cleaned_v2 AS
    SELECT {', '.join(columns_to_keep)}
    FROM euroStat_coal;
""")
cursor.execute("DROP TABLE euroStat_coal;")
conn.commit()

#### Step 5: Replace Invalid Values

The data contains invalid values (:), which are replaced with NULL values in SQL. This ensures that the dataset is clean for further analysis.

In [13]:
# Replace invalid values (':' -> NULL)
for column in columns_to_keep[1:]:  # Skip 'Country' column
    cursor.execute(f"""
        UPDATE euroStat_coal_cleaned_v2
        SET "{column}" = NULLIF("{column}", ':');
    """)
conn.commit()

#### Step 6: Rename Columns to Date Format

The column names are updated to represent proper dates (e.g., 2016-01, 2016-02, etc.). If the "Country" column does not exist in the cleaned table, it is added, and the final cleaned table is created with the new date-based column names.

In [14]:
# Rename columns with date formats (2016-01, 2016-02, ... , 2024-07)
date_columns = pd.date_range(start='2016-01', end='2024-07', freq='MS').strftime('%Y-%m').tolist()

# Ensure the columns_to_keep matches the new date columns
new_columns = ['Country'] + date_columns

# Check if "Country" column exists in the cleaned table
cursor.execute("PRAGMA table_info(euroStat_coal_cleaned_v2);")
columns_info = cursor.fetchall()
if "Country" not in [col[1] for col in columns_info]:
    print("Adding 'Country' column.")
    # Recreate the table with the 'Country' column
    cursor.execute("""
        CREATE TABLE euroStat_coal_cleaned_temp AS
        SELECT "TIME" AS Country, * FROM euroStat_coal_cleaned_v2;
    """)
    cursor.execute("DROP TABLE euroStat_coal_cleaned_v2;")
    cursor.execute("""
        CREATE TABLE euroStat_coal_cleaned_v2 AS
        SELECT Country, * FROM euroStat_coal_cleaned_temp;
    """)
    cursor.execute("DROP TABLE euroStat_coal_cleaned_temp;")
conn.commit()

Adding 'Country' column.


In [15]:
# Rename columns by recreating the table with the updated date-based column names
cursor.execute(f"""
    CREATE TABLE euroStat_coal_cleaned_final AS
    SELECT Country, {', '.join([f'"{old_col}" AS "{new_col}"' for old_col, new_col in zip(columns_to_keep[1:], date_columns)])}
    FROM euroStat_coal_cleaned_v2;
""")

In [16]:
# Drop the intermediate table and rename the final table back to the original cleaned table
cursor.execute("DROP TABLE euroStat_coal_cleaned_v2;")
cursor.execute("ALTER TABLE euroStat_coal_cleaned_final RENAME TO euroStat_coal;")
conn.commit()

#### Step 7: Query and Verify Data

Finally, the cleaned data is queried to verify that the transformations have been applied correctly. The first few rows of the cleaned data are displayed.

In [17]:
# Query the cleaned data
df_coal_cleaned = pd.read_sql_query("SELECT * FROM euroStat_coal LIMIT 5;", conn)
print(df_coal_cleaned)

    Country  2016-01  2016-02  2016-03  2016-04  2016-05  2016-06  2016-07  \
0   Belgium     2015     2014     2013     2012     2011     2010     2009   
1  Bulgaria     2015     2014     2013     2012     2011     2010     2009   
2   Czechia     2015     2014     2013     2012     2011     2010     2009   
3   Denmark     2015     2014     2013     2012     2011     2010     2009   
4   Germany     2015     2014     2013     2012     2011     2010     2009   

   2016-08  2016-09  ...  2023-10  2023-11  2023-12  2024-01  2024-02  \
0     2008     2007  ...     2013     2012     2011     2023     2022   
1     2008     2007  ...     2013     2012     2011     2023     2022   
2     2008     2007  ...     2013     2012     2011     2023     2022   
3     2008     2007  ...     2013     2012     2011     2023     2022   
4     2008     2007  ...     2013     2012     2011     2023     2022   

   2024-03  2024-04  2024-05  2024-06  2024-07  
0     2021     2020     2019     2018     2

#### Step 8: Close the SQLite Connection

Once all operations are complete, the SQLite connection is closed.

In [18]:
# Close the connection
conn.close()

## Workflow Summary

1.	Check and Create the SQLite Database:
•	The code first checks if the SQLite database already exists. If it does, the existing database is deleted to ensure a fresh start.
2.	Load Data from Excel into SQLite:
•	The coal energy data is loaded from an Excel file into a Pandas DataFrame and stored in the SQLite database.
3.	Remove the First Two Rows:
•	The first two rows of the dataset, which contain irrelevant information, are removed using SQL commands based on ROWID.
4.	Drop Unnecessary Columns:
•	Columns whose names start with ‘Unnamed’ are considered unnecessary and are removed. A new table is created with only the required columns.
5.	Replace Invalid Values:
•	Invalid values represented by ‘:’ are replaced with NULL to clean the data.
6.	Rename Columns to Date Format:
•	The column names are updated to reflect proper date formats (e.g., 2016-01, 2016-02), and if the Country column is missing, it is added.
7.	Query and Verify Data:
•	The cleaned data is queried from the SQLite database to ensure the transformations have been applied correctly.
8.	Close the SQLite Connection:
•	The connection to the SQLite database is closed once all operations are complete.


---

## Repeat for all datasets

### Individual databases

In [19]:
def process_energy_data_individual_db(file_paths):
    """
    This function processes the energy production data for multiple datasets (coal, geothermal, wind, etc.)
    by loading data from Excel, cleaning it, and saving it to individual SQLite databases.
    
    Parameters:
    - file_paths: Dictionary containing dataset names and file paths.
    """
    
    for dataset, file_path in file_paths.items():
        db_name = f'{dataset}_energy_data.db'  # Create a separate DB for each dataset
        
        # Step 1: Check if the database file exists, delete if it does
        if os.path.exists(db_name):
            print(f"Database {db_name} already exists. Deleting it...")
            os.remove(db_name)  # Delete the existing database file
            print(f"Database {db_name} has been deleted.")
        
        # Now create a new SQLite connection for this dataset
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()
        
        print(f"Processing dataset: {dataset}")
        
        # Step 2: Load data from Excel
        data = pd.read_excel(file_path, sheet_name=dataset, skiprows=range(0, 8))
        
        # Step 3: Store the raw data in the SQLite database
        data.to_sql(f'{dataset}', conn, if_exists='replace', index=False)
        
        # Step 4: Remove the first two rows
        cursor.execute(f"DELETE FROM {dataset} WHERE ROWID IN (1, 2);")
        conn.commit()
        
        # Step 5: Drop columns where the name starts with 'Unnamed'
        cursor.execute(f"PRAGMA table_info({dataset});")
        columns_info = cursor.fetchall()
        columns_to_keep = [col[1] for col in columns_info if not col[1].startswith('Unnamed')]
        
        # Create a new table with only the required columns
        cursor.execute(f"""
            CREATE TABLE {dataset}_cleaned_v2 AS
            SELECT {', '.join(columns_to_keep)}
            FROM {dataset};
        """)
        cursor.execute(f"DROP TABLE {dataset};")
        conn.commit()
        
        # Step 6: Replace invalid values (':' -> NULL)
        for column in columns_to_keep[1:]:  # Skip 'Country' column
            cursor.execute(f"""
                UPDATE {dataset}_cleaned_v2
                SET "{column}" = NULLIF("{column}", ':');
            """)
        conn.commit()
        
        # Step 7: Rename columns with date formats (2016-01, 2016-02, ..., 2024-07)
        date_columns = pd.date_range(start='2016-01', end='2024-07', freq='MS').strftime('%Y-%m').tolist()
        
        # Ensure the columns_to_keep matches the new date columns
        new_columns = ['Country'] + date_columns
        
        # Check if "Country" column exists in the cleaned table
        cursor.execute(f"PRAGMA table_info({dataset}_cleaned_v2);")
        columns_info = cursor.fetchall()
        if "Country" not in [col[1] for col in columns_info]:
            print("Adding 'Country' column.")
            # Recreate the table with the 'Country' column
            cursor.execute(f"""
                CREATE TABLE {dataset}_cleaned_temp AS
                SELECT "TIME" AS Country, * FROM {dataset}_cleaned_v2;
            """)
            cursor.execute(f"DROP TABLE {dataset}_cleaned_v2;")
            cursor.execute(f"""
                CREATE TABLE {dataset}_cleaned_v2 AS
                SELECT Country, * FROM {dataset}_cleaned_temp;
            """)
            cursor.execute(f"DROP TABLE {dataset}_cleaned_temp;")
        conn.commit()
        
        # Rename columns by recreating the table with the updated date-based column names
        cursor.execute(f"""
            CREATE TABLE {dataset}_cleaned_final AS
            SELECT Country, {', '.join([f'"{old_col}" AS "{new_col}"' for old_col, new_col in zip(columns_to_keep[1:], date_columns)])}
            FROM {dataset}_cleaned_v2;
        """)
        
        # Drop the intermediate table and rename the final table back to the original cleaned table
        cursor.execute(f"DROP TABLE {dataset}_cleaned_v2;")
        cursor.execute(f"ALTER TABLE {dataset}_cleaned_final RENAME TO {dataset};")
        conn.commit()

        # Step 8: Query and verify the cleaned data
        df_cleaned = pd.read_sql_query(f"SELECT * FROM {dataset} LIMIT 5;", conn)
        print(f"Cleaned data for {dataset}:")
        print(df_cleaned)
        
        # Close the connection after processing the current dataset
        conn.close()
        print(f"Finished processing {dataset}. Database saved as {db_name}.\n")

In [20]:
# Call the function to process the datasets and create individual databases
process_energy_data_individual_db(file_paths)

Database coal_energy_data.db already exists. Deleting it...
Database coal_energy_data.db has been deleted.
Processing dataset: coal
Adding 'Country' column.
Cleaned data for coal:
    Country  2016-01  2016-02  2016-03  2016-04  2016-05  2016-06  2016-07  \
0   Belgium     2015     2014     2013     2012     2011     2010     2009   
1  Bulgaria     2015     2014     2013     2012     2011     2010     2009   
2   Czechia     2015     2014     2013     2012     2011     2010     2009   
3   Denmark     2015     2014     2013     2012     2011     2010     2009   
4   Germany     2015     2014     2013     2012     2011     2010     2009   

   2016-08  2016-09  ...  2023-10  2023-11  2023-12  2024-01  2024-02  \
0     2008     2007  ...     2013     2012     2011     2023     2022   
1     2008     2007  ...     2013     2012     2011     2023     2022   
2     2008     2007  ...     2013     2012     2011     2023     2022   
3     2008     2007  ...     2013     2012     2011     202

## Workflow Summary

1.	Step 1: Check and Create the SQLite Database:
•	For each dataset (coal, wind, etc.), the function checks if a corresponding SQLite database already exists. If it does, the existing database file is deleted to ensure a clean slate.
2.	Step 2: Load Data from Excel:
•	The energy data from the respective Excel file (e.g., coal, wind, etc.) is loaded into a Pandas DataFrame.
3.	Step 3: Store Data in the SQLite Database:
•	The raw data is stored in an individual SQLite database for each dataset. For example, coal_energy_data.db, wind_energy_data.db, etc.
4.	Step 4: Remove the First Two Rows:
•	The first two rows, typically containing metadata or irrelevant headers, are removed using SQL commands.
5.	Step 5: Drop Unnecessary Columns:
•	Any columns with names starting with ‘Unnamed’ are dropped by selecting only the necessary columns.
6.	Step 6: Replace Invalid Values:
•	Invalid values (:) are replaced with NULL in SQL to clean the data.
7.	Step 7: Rename Columns to Date Format:
•	The columns are renamed to proper date formats (e.g., 2016-01, 2024-07). If a Country column is missing, it is added and the cleaned table is created with the updated column names.
8.	Step 8: Query and Verify:
•	The first few rows of the cleaned data are queried from each database to verify the cleaning steps.

---

## One database, multiple tables

In [21]:
def process_energy_data_single_db(file_paths):
    """
    This function processes energy production data for multiple datasets (coal, geothermal, wind, etc.)
    by loading data from Excel, cleaning it, and saving each dataset as a separate table in a single SQLite database.
    
    Parameters:
    - file_paths: Dictionary containing dataset names and file paths.
    """
    
    db_name = 'energySources.db'  # Single database for all energy sources
    
    # Step 1: Check if the database file exists, delete if it does
    if os.path.exists(db_name):
        print(f"Database {db_name} already exists. Deleting it...")
        os.remove(db_name)  # Delete the existing database file
        print(f"Database {db_name} has been deleted.")
    
    # Now create a new SQLite connection for the single database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    for dataset, file_path in file_paths.items():
        print(f"Processing dataset: {dataset}")
        
        # Step 2: Load data from Excel
        data = pd.read_excel(file_path, sheet_name=dataset, skiprows=range(0, 8))
        
        # Step 3: Store the raw data as a table in the SQLite database
        data.to_sql(f'{dataset}', conn, if_exists='replace', index=False)
        
        # Step 4: Remove the first two rows
        cursor.execute(f"DELETE FROM {dataset} WHERE ROWID IN (1, 2);")
        conn.commit()
        
        # Step 5: Drop columns where the name starts with 'Unnamed'
        cursor.execute(f"PRAGMA table_info({dataset});")
        columns_info = cursor.fetchall()
        columns_to_keep = [col[1] for col in columns_info if not col[1].startswith('Unnamed')]
        
        # Create a new table with only the required columns
        cursor.execute(f"""
            CREATE TABLE {dataset}_cleaned_v2 AS
            SELECT {', '.join(columns_to_keep)}
            FROM {dataset};
        """)
        cursor.execute(f"DROP TABLE {dataset};")
        conn.commit()
        
        # Step 6: Replace invalid values (':' -> NULL)
        for column in columns_to_keep[1:]:  # Skip 'Country' column
            cursor.execute(f"""
                UPDATE {dataset}_cleaned_v2
                SET "{column}" = NULLIF("{column}", ':');
            """)
        conn.commit()
        
        # Step 7: Rename columns with date formats (2016-01, 2016-02, ..., 2024-07)
        date_columns = pd.date_range(start='2016-01', end='2024-07', freq='MS').strftime('%Y-%m').tolist()
        
        # Ensure the columns_to_keep matches the new date columns
        new_columns = ['Country'] + date_columns

        # Check if "Country" column exists in the cleaned table
        cursor.execute(f"PRAGMA table_info({dataset}_cleaned_v2);")
        columns_info = cursor.fetchall()
        if "Country" not in [col[1] for col in columns_info]:
            print(f"Adding 'Country' column to dataset {dataset}.")
            # Recreate the table with the 'Country' column
            cursor.execute(f"""
                CREATE TABLE {dataset}_cleaned_temp AS
                SELECT "TIME" AS Country, * FROM {dataset}_cleaned_v2;
            """)
            cursor.execute(f"DROP TABLE {dataset}_cleaned_v2;")
            cursor.execute(f"""
                CREATE TABLE {dataset}_cleaned_v2 AS
                SELECT Country, * FROM {dataset}_cleaned_temp;
            """)
            cursor.execute(f"DROP TABLE {dataset}_cleaned_temp;")
        conn.commit()
        
        # Rename columns by recreating the table with the updated date-based column names
        cursor.execute(f"""
            CREATE TABLE {dataset}_cleaned_final AS
            SELECT Country, {', '.join([f'"{old_col}" AS "{new_col}"' for old_col, new_col in zip(columns_to_keep[1:], date_columns)])}
            FROM {dataset}_cleaned_v2;
        """)
        
        # Drop the intermediate table and rename the final table back to the original cleaned table
        cursor.execute(f"DROP TABLE {dataset}_cleaned_v2;")
        cursor.execute(f"ALTER TABLE {dataset}_cleaned_final RENAME TO {dataset}_cleaned;")
        conn.commit()

        # Step 8: Query and verify the cleaned data
        df_cleaned = pd.read_sql_query(f"SELECT * FROM {dataset}_cleaned LIMIT 5;", conn)
        print(f"Cleaned data for {dataset}:")
        print(df_cleaned)
        
    # Close the connection after processing all datasets
    conn.close()
    print(f"Finished processing all datasets. Database saved as {db_name}.\n")

In [22]:
# Call the function to process the datasets into a single database with multiple tables
process_energy_data_single_db(file_paths)

Database energySources.db already exists. Deleting it...
Database energySources.db has been deleted.
Processing dataset: coal
Adding 'Country' column to dataset coal.
Cleaned data for coal:
    Country  2016-01  2016-02  2016-03  2016-04  2016-05  2016-06  2016-07  \
0   Belgium     2015     2014     2013     2012     2011     2010     2009   
1  Bulgaria     2015     2014     2013     2012     2011     2010     2009   
2   Czechia     2015     2014     2013     2012     2011     2010     2009   
3   Denmark     2015     2014     2013     2012     2011     2010     2009   
4   Germany     2015     2014     2013     2012     2011     2010     2009   

   2016-08  2016-09  ...  2023-10  2023-11  2023-12  2024-01  2024-02  \
0     2008     2007  ...     2013     2012     2011     2023     2022   
1     2008     2007  ...     2013     2012     2011     2023     2022   
2     2008     2007  ...     2013     2012     2011     2023     2022   
3     2008     2007  ...     2013     2012     20

Let's check the number of tables inside the database:

In [24]:
# Assuming you already have a connection
db_name = 'energySources.db'

# Open a new connection if needed
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

# Step to check the tables inside the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Print the tables in the database
print("Tables in the database:")
for table in tables:
    print(table[0])

# Close the connection once done
conn.close()

Tables in the database:
coal_cleaned
nonRenewables_cleaned
renewables_cleaned
geothermal_cleaned
hydro_cleaned
naturalGas_cleaned
nuclear_cleaned
oil_cleaned
otherRenewables_cleaned
solar_cleaned
wind_cleaned


### Difference Between Individual Databases and Tables in a Single Database:

In this case, we created multiple tables inside a single SQLite database (energySources.db), with each energy source (coal, geothermal, etc.) being stored as a table within that database.

* **Advantages of using a single database with multiple tables**:

	* Simplified Querying: You can easily query data across different energy sources within a single connection.
	* Centralized Management: All data is stored in one file, making it easier to back up, manage, and share.
	* Easier Cross-Dataset Analysis: By having multiple tables in one database, you can perform SQL joins, unions, and other queries across different datasets more efficiently.

* **Advantages of Separate Databases**:

	* Data Isolation: Each dataset is fully isolated, which can be useful for security or performance reasons.
	* Smaller Files: Each database will be smaller, which might make it easier to handle backups and manage files separately.

In scenarios where you frequently need to analyze or compare data across different datasets (such as comparing coal and wind energy), having multiple tables inside a single database is more practical. If the datasets are independent or very large, separate databases might be preferred.

---

### Operations in DataBase

---

# SQL + Pandas

In this section we will cover a step-by-step example to integrate the energy production data stored in a database (SQLite) and then using Python to analyze and plot the data. This guide walks through each step, from loading CSV data into an SQLite database to querying and plotting the energy production data for different countries.

This sections includes downloading the data from individual `.csv` files, loading them into a SQLite database, and visualizing the energy production using Python.

This guide shows how to:
1. Download CSV files from Eurostat.
2. Load data into a SQLite database.
3. Use Python to query and visualize the data.

## Step 1: Download the Data from Eurostat

The datasets are available on the Eurostat website under [Energy Statistics](https://ec.europa.eu/eurostat/web/energy). Let’s assume the data files have been downloaded and stored in your `~/Downloads` folder as `.csv` files.

For this example, we are using the following datasets:

In [ ]:
import pandas as pd
import sqlite3
import os

In [ ]:
euroStat_coal_filePath = '../data/section4/euroStat/nrg_cb_pem_page_spreadsheet_coal.xlsx'
euroStat_nonRenewables_filePath = '../data/section4/euroStat/nrg_cb_pem_page_spreadsheet_combustionFuels_nonRenewables.xlsx'
euroStat_renewables_filePath = '../data/section4/euroStat/nrg_cb_pem_page_spreadsheet_combustionFuels_Renewables.xlsx'
euroStat_geothermal_filePath = '../data/section4/euroStat/nrg_cb_pem_page_spreadsheet_geothermal.xlsx'
euroStat_hydro_filePath = '../data/section4/euroStat/nrg_cb_pem_page_spreadsheet_hydro.xlsx'
euroStat_naturalGas_filePath = '../data/section4/euroStat/nrg_cb_pem_page_spreadsheet_naturalGas.xlsx'
euroStat_nuclear_filePath = '../data/section4/euroStat/nrg_cb_pem_page_spreadsheet_nuclear.xlsx'
euroStat_oil_filePath = '../data/section4/euroStat/nrg_cb_pem_page_spreadsheet_oil.xlsx'
euro_otherRenewables_filePath = '../data/section4/euroStat/nrg_cb_pem_page_spreadsheet_otherRenewables.xlsx'
euroStat_solar_filePath = '../data/section4/euroStat/nrg_cb_pem_page_spreadsheet_solar.xlsx'
euroStat_wind_filePath = '../data/section4/euroStat/nrg_cb_pem_page_spreadsheet_wind.xlsx'

## Step 2: Load Data into SQLite Database

We'll now read the downloaded CSV files, clean the data, and store them in an SQLite database.


### Python Code for Loading CSV Files into SQLite

In [ ]:
# Load .xlsx file
euroStat_coal = pd.read_excel(euroStat_coal_filePath, sheet_name='coal', skiprows=range(0, 8))
euroStat_nonRenewables = pd.read_excel(euroStat_nonRenewables_filePath, sheet_name='nonRenewables', skiprows=range(0, 8))
euroStat_renewables = pd.read_excel(euroStat_renewables_filePath, sheet_name='renewables', skiprows=range(0, 8))
euroStat_geothermal = pd.read_excel(euroStat_geothermal_filePath, sheet_name='geothermal', skiprows=range(0, 8))
euroStat_hydro = pd.read_excel(euroStat_hydro_filePath, sheet_name='hydro', skiprows=range(0, 8))
euroStat_naturalGas = pd.read_excel(euroStat_naturalGas_filePath, sheet_name='naturalGas', skiprows=range(0, 8))
euroStat_nuclear = pd.read_excel(euroStat_nuclear_filePath, sheet_name='nuclear', skiprows=range(0, 8))
euroStat_oil = pd.read_excel(euroStat_oil_filePath, sheet_name='oil', skiprows=range(0, 8))
euro_otherRenewables = pd.read_excel(euro_otherRenewables_filePath, sheet_name='otherRenewables', skiprows=range(0, 8))
euroStat_solar = pd.read_excel(euroStat_solar_filePath, sheet_name='solar', skiprows=range(0, 8))
euroStat_wind = pd.read_excel(euroStat_wind_filePath, sheet_name='wind', skiprows=range(0, 8))


In [ ]:
euroStat_coal.head()

,TIME,2016-01,Unnamed: 2,2016-02,Unnamed: 4,2016-03,Unnamed: 6,2016-04,Unnamed: 8,2016-05,...,Unnamed: 196,2024-03,Unnamed: 198,2024-04,Unnamed: 200,2024-05,Unnamed: 202,2024-06,Unnamed: 204,2024-07
0,GEO (Labels),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,European Union - 27 countries (from 2020),:,NaN,:,NaN,:,NaN,:,NaN,:,...,NaN,23110.523,NaN,:,NaN,:,NaN,:,NaN,:
2,Belgium,:,NaN,:,NaN,:,NaN,:,NaN,:,...,NaN,160.492,NaN,168.435,NaN,179.131,NaN,:,NaN,:
3,Bulgaria,:,NaN,:,NaN,:,NaN,:,NaN,:,...,NaN,519.203,NaN,360.31,NaN,290.517,NaN,:,NaN,:
4,Czechia,:,NaN,:,NaN,:,NaN,:,NaN,:,...,NaN,2159.708,NaN,1550.964,NaN,1480.491,NaN,:,NaN,:


In [ ]:
# Connect to SQLite database (it will be created if it doesn't exist)
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

In [ ]:
# Add the euroStat_coal dataset to the database using SQL
euroStat_coal.to_sql('euroStat_coal', conn, if_exists='replace', index=False)

50

In [3]:
# Define file paths and dataset names
datasets = ['coal', 'nonRenewables', 'renewables', 'geothermal', 'hydro', 'naturalGas', 'nuclear', 'oil', 'otherRenewables', 'solar', 'wind']
data_path = os.path.expanduser('~/Downloads/')  # Path where the CSV files are downloaded
db_path = 'energy_data.db'  # SQLite database file

In [ ]:



for dataset in datasets:
    # Load the CSV file into a pandas DataFrame
    csv_file = os.path.join(data_path, f'{dataset}.csv')
    df = pd.read_csv(csv_file, skiprows=1)  # Assuming the first row is metadata

    # Rename columns properly (example: replace unnamed columns with actual month names)
    df.columns = ['Country'] + [f'Month_{i+1}' for i in range(len(df.columns)-1)]

    # Store each DataFrame in a table named after the dataset (e.g., 'wind', 'geothermal')
    df.to_sql(dataset, conn, if_exists='replace', index=False)

    print(f"Data from {dataset}.csv loaded into the {dataset} table.")

# Commit and close the database connection
conn.commit()
conn.close()

### Explanation:
1. **Loading CSV Files**: We read each CSV file and load it into a pandas DataFrame, assuming the first row is metadata.
2. **Cleaning Data**: We rename the columns, as some columns might be unnamed.
3. **Storing in SQLite**: Each dataset is saved as a table in the SQLite database. For example, the `wind.csv` file will be saved in a table named `wind`.

## Step 3: Query and Analyze the Data

Now that we’ve stored the data in an SQLite database, we can query it using SQL and visualize it in Python. Let’s start by analyzing and plotting the wind energy production data.

### Python Code for Querying and Plotting Wind Data

In [ ]:
# Reconnect to the SQLite database
conn = sqlite3.connect('energy_data.db')

# Query wind energy data
wind_query = """
SELECT * FROM wind;
"""

# Load the data into a pandas DataFrame
wind_data = pd.read_sql_query(wind_query, conn)

# Close the database connection
conn.close()

# Preview the data
print(wind_data.head())

# Example plotting wind energy production for Belgium
country = 'Belgium'

# Extract the data for Belgium (replace : and NaN values with 0 for simplicity)
wind_belgium = wind_data[wind_data['Country'] == country].fillna(0)

# Transpose to have months on x-axis
months = wind_belgium.columns[1:]  # Skip the 'Country' column
energy_values = wind_belgium.iloc[0, 1:]  # Skip the 'Country' column

# Plotting the data
plt.figure(figsize=(10, 6))
plt.plot(months, energy_values, marker='o')
plt.title(f'Wind Energy Production in {country} (MWh)')
plt.xlabel('Months')
plt.ylabel('Energy Generated (MWh)')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

### Explanation:
1. **Query Data**: We query the `wind` table and load the data into a pandas DataFrame.
2. **Data Cleaning**: We fill missing values (`NaN` or `:` in the dataset) with `0` for simplicity.
3. **Plotting**: We plot the wind energy production for Belgium over the months using `matplotlib`.

## Step 4: Plotting Energy Production for Multiple Countries

You can extend this example to plot data for multiple countries or compare the production of different renewable energy sources.

### Example Code to Plot Energy Production for Multiple Countries

In [ ]:
# Example for comparing wind energy production of Belgium, Germany, and Spain
countries = ['Belgium', 'Germany', 'Spain']
plt.figure(figsize=(10, 6))

for country in countries:
    country_data = wind_data[wind_data['Country'] == country].fillna(0)
    energy_values = country_data.iloc[0, 1:]  # Skip the 'Country' column
    
    plt.plot(months, energy_values, marker='o', label=country)

# Add labels, title, and legend
plt.title('Wind Energy Production in Belgium, Germany, and Spain (MWh)')
plt.xlabel('Months')
plt.ylabel('Energy Generated (MWh)')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### Explanation:
1. **Multiple Countries**: We loop through a list of countries (e.g., Belgium, Germany, Spain) and plot their wind energy production on the same graph for comparison.

## Workflow Summary

1. **Downloading and Loading Data**: We downloaded the CSV files from Eurostat and loaded them into an SQLite database.
2. **Querying the Database**: We queried specific tables from the database using SQL queries in Python.
3. **Visualizing the Data**: We plotted the renewable energy production data (wind in this case) using matplotlib to visualize the monthly energy production for different countries.

This approach allows you to easily handle large datasets, store them in a database, and perform complex analysis and visualization using Python.


## Summary

In this section, we provided a detailed step-by-step guide to integrating renewable energy production data stored in an SQLite database with Python for data analysis and visualization. The process includes downloading CSV files from the Eurostat website, loading them into an SQLite database using Python, and then querying the data for analysis and visualization.

## Key Points Covered:

1.	Data Download and Preparation: We downloaded the relevant CSV files for different renewable energy sources (wind, solar, geothermal, hydro, etc.) from Eurostat and stored them locally.
2.	Loading Data into SQLite: The downloaded CSV files were cleaned and stored in an SQLite database using the pandas and sqlite3 libraries in Python. Each dataset was stored in its own table within the database.
3.	Querying the Data: We queried the SQLite database for specific data (e.g., wind energy production) using SQL commands within Python.
4.	Visualizing Data: Using matplotlib, we created plots of the energy production data (e.g., wind energy production in Belgium) across different months.
5.	Comparison Across Countries: We extended the example to compare the energy production of multiple countries on a single plot, allowing for better insights into trends and differences in renewable energy generation.

## Lessons Learned

1.	**Handling Large Datasets**: By using `SQLite`, we can efficiently store and manage large datasets locally, allowing for easy querying and retrieval of data as needed.
2.	**Data Cleaning and Preparation**: Properly cleaning and formatting the data, such as renaming columns and filling missing values, is essential for accurate analysis.
3.	**SQL Integration with Python**: Using `SQL` within Python allows for powerful querying capabilities, which can be further extended by combining results with pandas for in-depth analysis.
4.	**Visualization**: Python’s `matplotlib` library makes it easy to create visual representations of the data, helping to uncover trends and insights from complex datasets.
5.	**Scalability**: The approach can be easily extended to more complex datasets, multiple energy sources, or multiple countries, providing a scalable solution for energy data analysis.